### Imports

In [1]:
from scripts import eng_processor
from scripts.text_preprocessing import TextImageDataset
from scripts.train_model import BertEncoder, Generator, Discriminator
from scripts.inference import generate_image_from_text
from scripts.utils import combine_dataset, Evaluator

from torchvision import transforms
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.optim as optim
from PIL import Image
from tqdm import tqdm
import matplotlib.pyplot as plt
from pycocotools.coco import COCO
import os
from collections import defaultdict
import pandas as pd

/opt/anaconda3/envs/intro_dl/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# checking for mac gpu which should print mps
if torch.backends.mps.is_available():
    device = torch.device("mps")  # Apple Metal GPU
elif torch.cçuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

print(device)

mps


### Text preprocessing

In [3]:
combine_dataset()

loading annotations into memory...
Done (t=0.33s)
creating index...
index created!
✅ Combined dataset saved with 90488 entries (COCO capped at 10000)


In [4]:
combined_df = pd.read_csv("datasets/combined_dataset.csv")
combined_df.tail(5)

,image_path,caption,source
90483,datasets/coco_dataset/train2017/000000370808.jpg,A black cat looking out the window at a black ...,MSCOCO
90484,datasets/coco_dataset/train2017/000000370808.jpg,Black cat sitting on window ledge looking outs...,MSCOCO
90485,datasets/coco_dataset/train2017/000000370808.jpg,A black cat looks out the window as a crow out...,MSCOCO
90486,datasets/coco_dataset/train2017/000000370808.jpg,A cat by a window with a small bird outside.,MSCOCO
90487,datasets/coco_dataset/train2017/000000370808.jpg,A cat watches a bird through a window.,MSCOCO


In [5]:
# def load_text_annotations(text):

#     mapp = {}
    
#     for line in text.split("\n"):

#         token = line.split("\t")

#         if len(line) < 3:
#             continue

#         # image id 
#         img_id = token[0].split('.')[0]
#         # image description
#         img_des = token[1] 

#         if img_id not in mapp: 
#             mapp[img_id] = list() 
#         mapp[img_id].append(img_des)

#     return mapp

In [6]:
# image_to_text_map = load_text_annotations(text_annot)

In [7]:
# image_to_text_map["2097420505_439f63c863"]

In [8]:
# import nltk
# import ssl

# try:
#     _create_unverified_https_context = ssl._create_unverified_context
# except AttributeError:
#     pass
# else:
#     ssl._create_default_https_context = _create_unverified_https_context

# nltk.download('stopwords')  

In [9]:
image_to_captions = defaultdict(list)

for _, row in combined_df.iterrows():
    img_path = row['image_path']
    image_to_captions[img_path].append(row['caption'])

In [10]:
image_paths = list(image_to_captions.keys())
train_ids, test_ids = train_test_split(image_paths, test_size=0.2, random_state=42)


In [11]:
print(f"Train samples: {len(train_ids)}")
print(f"Test samples: {len(test_ids)}")


Train samples: 14473
Test samples: 3619




### create dataloader

In [12]:
# image transform
transform = transforms.Compose([
    transforms.Resize((256, 256)),  # Resize the image
    transforms.ToTensor(),  # Convert PIL Image to tensor
    transforms.Lambda(lambda x: (x * 2) - 1)  # Scale from [0,1] to [-1,1]
])


train_dataset = TextImageDataset(
    image_to_captions=image_to_captions,
    image_paths=train_ids,  
    transform=transform
)

test_dataset = TextImageDataset(
    image_to_captions=image_to_captions,
    image_paths=test_ids,  
    transform=transform
)

# Create dataloaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

Dataset size: 72388 (all captions included)
Dataset size: 18095 (all captions included)


## Train model

In [13]:
# initialize encoder decoder and discriminator

text_encoder =  BertEncoder().to(device)
generator =  Generator().to(device)
discriminator = Discriminator().to(device)

In [14]:
# !pip install 'torchmetrics[image]' git+https://github.com/openai/CLIP.git

#### Initialize optimizers

In [15]:
adv_loss = nn.BCELoss()

# optimizer for generator
optimizer_G = optim.Adam(
    list(generator.parameters()) + list(text_encoder.parameters()),
    lr=0.0002, betas=(0.5, 0.999)
)


# optimizer for discriminator

optimizer_D = optim.Adam(discriminator.parameters(), lr=0.0001, betas=(0.5, 0.999))

evaluator = Evaluator(device)

/opt/anaconda3/envs/intro_dl/lib/python3.12/site-packages/torchmetrics/utilities/prints.py:43: UserWarning: Metric `InceptionScore` will save all extracted features in buffer. For large datasets this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)
Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


In [16]:
# Assuming real_images, text_embedding, valid, and fake are already defined
num_epochs =50
for epoch in range(num_epochs):
    # Training 
    generator.train()
    discriminator.train()

    for i, batch in enumerate(tqdm(train_loader, desc=f"Training Epoch {epoch+1}")):
        # Skip batches with None values (if any fallthrough from dataset)
        if None in batch.values():
            continue
        
        real_images = batch['image'].to(device)
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)

        valid = torch.ones((real_images.size(0), 1), requires_grad=False).to(device)
        fake = torch.zeros((real_images.size(0), 1), requires_grad=False).to(device)

        # Train generator first
        optimizer_G.zero_grad()
        
        text_embedding = text_encoder(input_ids, attention_mask)
        noise = torch.randn(real_images.size(0), 100, device=device)
        
        gen_images = generator(text_embedding, noise)
        g_loss = adv_loss(discriminator(gen_images, text_embedding), valid)
        
        g_loss.backward()
        optimizer_G.step()

        # Train discriminator
        optimizer_D.zero_grad()
        real_loss = adv_loss(discriminator(real_images, text_embedding.detach()), valid)
        fake_loss = adv_loss(discriminator(gen_images.detach(), text_embedding.detach()), fake)
        
        d_loss = (real_loss + fake_loss) / 2

        d_loss.backward()
        optimizer_D.step()

        if i % 100 == 0:
            print(f"[Batch {i}/{len(train_loader)}] [D loss: {d_loss.item():.4f}] [G loss: {g_loss.item():.4f}]")

    # Evaluate using evaluator
    generator.eval()
    evaluator.reset()

    with torch.no_grad():
        for batch in test_loader:
            real_images = batch["image"].to(device)
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            captions = batch["caption_text"]

            # Generate images
            text_embedding = text_encoder(input_ids, attention_mask)
            noise = torch.randn(real_images.size(0), 100).to(device)

            fake_images = generator(text_embedding, noise)

            metrics = evaluator.compute_metrics(
                real_images,
                fake_images,
                captions
            )

        print(f"Epoch {epoch+1}/{num_epochs}")
        print(f"FID: {metrics['fid']:.2f}")
        print(f"Inception Score: {metrics['inception_score']:.2f}")
        print(f"CLIP Score: {metrics['clip_score']:.2f}")
        print("---------------------")

        # Save the model checkpoints to use later
        if (epoch + 1) % 10 == 0:
            torch.save({
                'generator': generator.state_dict(),
                'discriminator': discriminator.state_dict(),
                'text_encoder': text_encoder.state_dict(),
                'metrics': metrics
            }, f"checkpoints_epoch{epoch+1}.pt")

Training Epoch 1:   0%|                        | 1/2263 [00:01<41:40,  1.11s/it]

[Batch 0/2263] [D loss: 0.6720] [G loss: 0.6801]


Training Epoch 1:   4%|▉                     | 101/2263 [01:02<25:37,  1.41it/s]

[Batch 100/2263] [D loss: 0.2334] [G loss: 8.6559]


Training Epoch 1:   9%|█▉                    | 201/2263 [02:06<24:52,  1.38it/s]

...

[Batch 200/2263] [D loss: 0.0000] [G loss: 21.1080]


Training Epoch 11:  13%|██▊                  | 301/2263 [03:11<24:21,  1.34it/s]

[Batch 300/2263] [D loss: 0.0000] [G loss: 21.0419]


Training Epoch 11:  16%|███▍                 | 365/2263 [03:53<20:15,  1.56it/s]


KeyboardInterrupt: 

In [ ]:
text_prompt = "man standing"
generated_image = generate_image_from_text(
    text_prompt, 
    text_encoder, 
    generator, 
    device
)

plt.imshow(generated_image)
plt.axis('off')
plt.title(text_prompt)
plt.show()

## Evaluate on test data